### Step 1

Initialize the notebook, loading the configuration and importing libraries.

In [ ]:
# Import the libraries required for the notebook
import asyncio

# Instatiate the config class
import sys
sys.path.append('..')
from config.notebook_config import *
config = notebook_config()
# Load config from file
config.load_config_from_file()


### Step 2

Load the files from a folder and create a dataframe with the data.

In [ ]:
# Load pdf files from the data folder (these will be invoices)
from os import listdir
from os.path import isfile, join
mypath = "<path to your pdf files>"

pdf_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# Print the full path of the files
from pathlib import Path
pdf_files = [str(Path(mypath) / f) for f in pdf_files]


In [ ]:
import semantic_kernel
from plugins.form_recognizer.sk_form_recognizer import FormRecognizer
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = semantic_kernel.Kernel()
print("Kernel loaded.")

deployment = config.model
endpoint = config.endpoint
api_key = config.azure_api_key
kernel.add_chat_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
print(f"Fire-up the kernel with {deployment}")

form_recognizer = kernel.import_skill(FormRecognizer(config.get_form_regonizer_endpoint(), config.get_form_regonizer_key()))
print("Plugins ready.")

In [ ]:
plugins_directory = "../plugins"

# Import the semantic functions
my_functions = kernel.import_semantic_skill_from_directory(plugins_directory, "form_recognizer")

print("Plugins loaded.")

In [ ]:
result = await kernel.run_async(
    form_recognizer["process_invoice"],
    input_str=pdf_files[0],
)

# Result is a json file with the information extracted from the invoice, print in pretty format
import json
# Load the JSON string into a Python object
result_obj = json.loads(result.result)
# Pretty print the object
print(json.dumps(result_obj, indent=4, sort_keys=True, ensure_ascii=False))

In [ ]:
# Summarize the invoice
input_str = result.result
invoice_summary = asyncio.create_task(kernel.run_async(my_functions["summarize"], input_str=input_str))
invoice_summary_result = await invoice_summary
print(invoice_summary_result.result)